In [1]:
%%bash
python3 -m pip install kafka-python

In [2]:
import json
from kafka import KafkaProducer
from kafka.errors import KafkaError

producer = KafkaProducer(bootstrap_servers=['10.10.139.63:9092'],
                         value_serializer=lambda v: json.dumps(v).encode('utf-8'))

def sendRecord(rdd):
    records = rdd.collect()
    for r in records:
        producer.send('output', r)
        producer.flush()

In [3]:
import os
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-streaming-kafka-0-8_2.11:2.4.3 pyspark-shell'

from pyspark import SparkContext
import json

sc = SparkContext()
sc.setLogLevel("WARN")

from pyspark.streaming import StreamingContext
from pyspark.streaming.kafka import KafkaUtils

ssc = StreamingContext(sc, 30)

directKafkaStream = KafkaUtils.createDirectStream(ssc, ['test'], {"metadata.broker.list": '10.10.139.63:9092'}, 
                                                 messageHandler=lambda x: json.loads(x.message))
# Remove message without a 'ts_ingest' field or empty timestamps
filtered = directKafkaStream.filter(lambda x: 'ts_ingest' in x.keys() and x['ts_ingest'])
filtered.foreachRDD(sendRecord)

In [ ]:
ssc.start()
ssc.awaitTermination()